In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kriti-2024/sample_submission.csv
/kaggle/input/kriti-2024/train.csv
/kaggle/input/kriti-2024/test.csv
/kaggle/input/scibert/scibert/config.json
/kaggle/input/scibert/scibert/model.safetensors


In [2]:
!pip install tensorflow_addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.1.5
    Uninstalling typeguard-4.1.5:
      Successfully uninstalled typeguard-4.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.6.4 requires typeguard<5,>=4.1.2, but you have typeguard 2.13.3 which is incompatible.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
from imblearn.ensemble import EasyEnsembleClassifier
import numpy as np
import tensorflow_addons as tfa
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import tensorflow_addons as tfa

2024-02-09 08:40:33.219979: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 08:40:33.220109: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 08:40:33.368304: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our Tens

In [4]:
df=pd.read_csv('/kaggle/input/kriti-2024/train.csv')
test_df=pd.read_csv('/kaggle/input/kriti-2024/test.csv')

In [5]:
X = df[['Title', 'Abstract']]
y = df['Categories']

In [6]:
mlb = MultiLabelBinarizer()
y_binary = mlb.fit_transform(y.apply(eval))


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.15, random_state=83)

In [8]:
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Remove stop words
    text = ' '.join([word for word in text.split() if word not in stop_words])
        
    return text

In [9]:
X_train_processed = [preprocess_text(title + ': ' + abstract) for title, abstract in zip(X_train['Title'], X_train['Abstract'])]
X_test_processed = [preprocess_text(title + ': ' + abstract) for title, abstract in zip(X_test['Title'], X_test['Abstract'])]

In [10]:
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased', do_lower_case=True)


vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

In [11]:
def tokenize_text(text):
    return tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt', padding='max_length', truncation=True, max_length=512)

In [12]:
X_train_tokens = [tokenize_text(i) for i in X_train_processed]
X_test_tokens = [tokenize_text(i) for i in X_test_processed]
X_train_attention_masks = torch.cat([x['attention_mask'] for x in X_train_tokens], dim=0)
X_test_attention_masks= torch.cat([x['attention_mask'] for x in X_test_tokens], dim=0)

In [13]:
X_train_inputs = torch.cat([x['input_ids'] for x in X_train_tokens], dim=0)
X_test_inputs = torch.cat([x['input_ids'] for x in X_test_tokens], dim=0)

In [14]:
model_name = 'allenai/scibert_scivocab_uncased'
scibert_model = BertForSequenceClassification.from_pretrained('/kaggle/input/scibert/scibert')


In [15]:
train_dataset = TensorDataset(X_train_inputs, X_train_attention_masks, torch.tensor(y_train, dtype=torch.float32))
train_dataloader = DataLoader(train_dataset, batch_size=9, shuffle=True)

In [16]:
from tqdm import tqdm
import torch

# Move model to GPU
scibert_model.to('cuda')

optimizer = AdamW(scibert_model.parameters(), lr=3e-5)
epochs = 1
scibert_model.train()

for epoch in range(epochs):
    epoch_loss = 0.0
    with tqdm(total=len(train_dataloader), desc=f'Epoch {epoch + 1}/{epochs}', unit='batch') as pbar:
        for batch in train_dataloader:
            inputs = batch[0].to('cuda')
            attention_masks = batch[1].to('cuda')
            labels = batch[2].to('cuda')
            optimizer.zero_grad()
            outputs = scibert_model(inputs,attention_mask=attention_masks, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            pbar.set_postfix({'Loss': epoch_loss / (pbar.n + 1e-12)})
            pbar.update(1)


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/1: 100%|██████████| 4837/4837 [37:56<00:00,  2.12batch/s, Loss=0.017]


In [17]:
batch_size = 9  # Adjust the batch size based on your available GPU memory

# Move the test data to CUDA
X_test_inputs = X_test_inputs.to('cuda')
X_test_attention_masks = X_test_attention_masks.to('cuda')

# Split the test data into batches
X_test_inputs_batches = torch.split(X_test_inputs, batch_size)
X_test_attention_masks_batches = torch.split(X_test_attention_masks, batch_size)
# Make predictions on the test data
y_pred_list = []

scibert_model.eval()
with torch.no_grad():
    for inputs_batch, attention_masks_batch in zip(X_test_inputs_batches, X_test_attention_masks_batches):
        inputs_batch = inputs_batch.to('cuda')
        attention_masks_batch = attention_masks_batch.to('cuda')

        y_pred_batch = scibert_model(inputs_batch, attention_mask=attention_masks_batch)[0].sigmoid().cpu().numpy()
        y_pred_list.append(y_pred_batch)

# Concatenate predictions from all batches
y_pred = np.concatenate(y_pred_list, axis=0)

# Convert predicted probabilities to binary predictions
y_pred_classes = (y_pred > 0.5).astype(int)

# Calculate Macro F1 score
f1 = f1_score(y_test, y_pred_classes, average='macro')
print("Macro F1 Score on Test Set:", f1)


Macro F1 Score on Test Set: 0.6777930293549181


In [18]:
# Preprocess the test data
test_df['Processed_Text'] = [preprocess_text(title + ' ' + abstract) for title, abstract in zip(test_df['Title'], test_df['Abstract'])]

# Tokenize and preprocess the text data for test
X_test_tokens1 = [tokenize_text(text) for text in test_df['Processed_Text']]
X_test_inputs1 = torch.cat([x['input_ids'] for x in X_test_tokens1], dim=0)
X_test_attention_masks1 = torch.cat([x['attention_mask'] for x in X_test_tokens1], dim=0)

# Move test data to GPU
X_test_inputs1 = X_test_inputs1.to('cuda')
X_test_attention_masks1 = X_test_attention_masks1.to('cuda')

# Set batch size
batch_size = 9

# Split the test data into batches
X_test_inputs_batches1 = torch.split(X_test_inputs1, batch_size)
X_test_attention_masks_batches1= torch.split(X_test_attention_masks1, batch_size)

# Make predictions on the test data
y_pred_list = []

scibert_model.eval()
with torch.no_grad():
    with tqdm(total=len(X_test_inputs_batches1), desc='Predicting', unit='batch') as pbar:
        for inputs_batch, attention_masks_batch in zip(X_test_inputs_batches1, X_test_attention_masks_batches1):
            inputs_batch = inputs_batch.to('cuda')
            attention_masks_batch = attention_masks_batch.to('cuda')

            y_pred_batch = scibert_model(inputs_batch, attention_mask=attention_masks_batch)[0].sigmoid().cpu().numpy()
            y_pred_list.append(y_pred_batch)

            pbar.update(1)

# Concatenate predictions from all batches
y_pred = np.concatenate(y_pred_list, axis=0)

# Convert predicted probabilities to binary predictions
y_pred_classes = (y_pred > 0.5).astype(int)

# Create a DataFrame with columns matching mlb.classes_
predictions_df = pd.DataFrame(y_pred_classes, columns=mlb.classes_)

# Concatenate the predictions DataFrame with the original test_df
final_df = pd.concat([test_df, predictions_df], axis=1)

# Display or save the final_df as needed
print(predictions_df)
final_df.to_csv('/kaggle/working/finaldf.csv')

Predicting: 100%|██████████| 1220/1220 [03:11<00:00,  6.39batch/s]


       cs.AI  cs.AR  cs.CE  cs.CL  cs.CR  cs.CV  cs.DB  cs.DC  cs.DM  cs.GT  \
0          0      0      0      0      0      0      0      0      0      0   
1          0      1      0      0      0      0      0      0      0      0   
2          0      0      0      0      0      1      0      0      0      0   
3          0      1      0      0      0      1      0      0      0      0   
4          0      0      0      0      0      0      0      0      0      0   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
10969      0      0      0      0      0      0      0      0      0      0   
10970      0      0      0      1      0      0      0      0      0      0   
10971      0      0      0      0      0      1      0      0      0      0   
10972      1      0      0      0      0      0      0      0      0      0   
10973      0      0      0      0      0      0      0      0      0      0   

       ...  q-fin.MF  q-fin.PM  q-fin.PR  q-fin.RM 

In [19]:
file1_path = '/kaggle/input/kriti-2024/sample_submission.csv'
df1 = pd.read_csv(file1_path)
column_order = df1.columns

# Reorder columns in the second DataFrame based on the order of the first DataFrame
df2_reordered = final_df[column_order]

# Save the modified DataFrame to a new CSV file or overwrite the existing one
output_path = '/kaggle/working/submission9_reordered.csv'
df2_reordered.to_csv(output_path, index=False)

# Optionally, you can also display the modified DataFrame
print(df2_reordered)

          Id  math.AT  stat.AP  cs.AR  math.QA  q-bio.MN  eess.AS  eess.IV  \
0      30332        0        0      0        0         0        0        0   
1      50337        0        0      1        0         0        0        0   
2      66515        0        0      0        0         0        0        1   
3      57464        0        0      1        0         0        0        1   
4      43169        0        0      0        0         0        0        0   
...      ...      ...      ...    ...      ...       ...      ...      ...   
10969  41708        0        0      0        0         0        0        0   
10970  38843        0        0      0        0         0        0        0   
10971  57571        0        0      0        0         0        0        0   
10972  31964        0        0      0        0         0        0        0   
10973  22229        0        0      0        0         0        0        0   

       stat.ME  econ.GN  ...  cs.SE  cs.RO  stat.TH  cs.DB  mat

In [20]:
# Save the trained SciBERT model
scibert_model.save_pretrained('/kaggle/working/scibert_trained')